# Introduction to Quantum Monte Carlo methods

Monte Carlo methods are extremely powerful tools to deal with problems that have vast phase spaces, such as quantum many-body systems. Here, we will focus on Quantum Monte Carlo (QMC) methods using a specific technique, called Variational Quantum Monte Carlo (VQMC). VQMC is what we find to be the most intuitive and simple, while already allowing us to dive into the depths of QMC methods.

## Monte Carlo Integration

The main power of Monte Carlo methods comes from the capability of computing high-dimensional integrals in large spaces. In physics, this allows us to compute expectation values of the form $$\langle f\rangle = \int dx p(x)f(x) \ \ \ \text{or} \ \ \ \langle f \rangle = \sum_{x} p(x)f(x)$$ for continuous and discrete sytems, respectively. 

Physics is benevolent and, generally, the systems of interest only span a tiny bit of their phase space, meaning that $p(x)\simeq 0$ for most states $x$ and, therefore, most of the terms in the previous sum have a meaningless contribution. With Monte Carlo, rather than summing over all the possible states $x$, we approximate the expectation value by sampling from $p(x)$. Hence, $$\langle f \rangle \approx \frac{1}{N}\sum_{i=1}^Nf(x_i),$$ where $x_i$ are sampled according to $p(x)$. This is called importance sampling and it allows us to obtain reasonably good approximations with a limitted amount of samples.

### Energy expectation of a quantum system

In quantum phsyics, a quantity of utmost interest is the expected value of the energy of a system under the action of a Hamiltonian $H$ and a wave function $\Psi(x)$ $$\langle H \rangle = \frac{\langle\Psi^*|H|\Psi\rangle}{\langle \Psi|\Psi\rangle} = \frac{\int dx\Psi^*(x)H\Psi(x)}{\int dx\Psi^*(x)\Psi(x)}.$$

From now on, we will omit the dependency on the state and denote $\Psi\equiv\Psi(x)$ unless needed for clarification. By introducing a term $\frac{\Psi}{\Psi}$ into the numerator, we can rewrite the integral in a convenient way for Monte Carlo integration $$\langle H \rangle = \frac{\int \Psi^*\frac{\Psi}{\Psi}H\Psi}{\int \Psi^*\Psi} = \frac{\int |\Psi|^2 \frac{H\Psi}{\Psi}}{\int |\Psi|^2} = \int \rho E_L,$$
where $\rho=\frac{|\Psi|^2}{\int|\Psi|^2}$ is the probability density and $E_L=\frac{H\Psi}{\Psi}$ is the so called **local energy**. 

Hence, the expected energy can be computed via Monte Carlo integration as the expectation value of the local energy over the probability distribution $\rho=\frac{|\Psi|^2}{\int|\Psi|^2}$, such that $$\langle H\rangle \approx \frac{1}{N}\sum_{k=1}^NE_L(x_k)$$

## Importance Sampling

One of the most important aspects for Monte Carlo integration is the way that importance sampling is done. Markov Chain Monte Carlo (MCMC) is an efficient approach to perform sampling in many dimensions when the probability density $p(x)$ is dominated by a small part of the whole state space. 

Samples are drawn iteratively forming a Markov Chain starting from any given state. In order to properly compute the expected value, the Markov Chain needs to converge to the **stationary distribution** $p(x)$ regardless of the initial state. 

Let $t(x\rightarrow x')$ be the probability to transition from state $x$ to $x'$ such that $\sum_{x'}t(x\rightarrow x')=1$, and $p_s(x)$ the probability to be in state $x$ at step $s$. Then, $p_{s+1}(x) = \sum_{x'}p_s(x')t(x'\rightarrow x)$. A stationary probability is obtained when $p_s(x)$ is independent of the step and, therefore, $$p(x) = \sum_{x'}p(x')t(x'\rightarrow x).$$ 

If a Markov chain is irreducible, the stationary distribution is unique and, if it is also aperiodic, it converges to it. A sufficient condition for stationarity is satsifying the detailed balance condition $p(x)t(x\rightarrow x') = p(x')t(x'\rightarrow x)$.

The **Metropolis-Hastings** algorithm is known to satisfy detailed balance. This is a very simple algorithm in which we split the transition probability $t(x\rightarrow x')$ into two factors: the probability to propose the next state $c(x\rightarrow x')$ and the probability to accept the next state $a(x\rightarrow x')$ such that $$t(x\rightarrow x') = c(x\rightarrow x')a(x\rightarrow x').$$ Detail balanced is fulfilled by taking $a(x\rightarrow x')=\min\left\{1, \frac{p(x')}{p(x)}\frac{c(x'\rightarrow x)}{c(x\rightarrow x')}\right\}$.

Generally, the probability to propose a state is symmetric $c(x\rightarrow x')=c(x'\rightarrow x)$, as it can be, for instance, the case of randomly flipping a spin in a lattice. Therefore, $$a(x\rightarrow x') = \min\left\{1, \frac{p(x')}{p(x)}\right\}.$$

A Markov Chain is generated by iterating over the following two steps:
1. With a state $x$, propose a new state $x'$ with probability $c(x\rightarrow x')$
2. Accept $x'$ with probability $a(x\rightarrow x')$. If rejected, the next state is $x$.

The time it takes for the Markov Chain to converge to the stationary distribution is called **thermalisation**. In other words, thermalisation time is the time it takes to the Markov Chain to forget its initial state. With MCMC we need to wait for the thermalisation to finish before we can start drawing samples from the desired probability distribution. These samples, though, will be highly correlated between one another, thus requiring careful error analysis to be properly handled. We will deal with this later on.  

### Metropolis-Hastings for quantum problems

As was previously introduced, the expectation value of the energy can be obtained by sampling configurations according to the distribution $\rho=\frac{|\Psi|^2}{\int|\Psi|^2}$. With the Markov Chain we want to converge to this distribution and, therefore, the acceptance probabilities need to be accordingly defined $$a(x\rightarrow x') = \min\left\{
1, \frac{\rho(x')}{\rho(x)}\right\} = \min\left\{1, \frac{|\Psi(x')|^2}{|\Psi(x)|^2}\right\}.$$ Notice that the normalisation factor $\int|\Psi|^2$ cancels out and, thus, we never have to worry about normalising the probabilities, which would, most times, make it intractable.


## Example

Right now we have the tools to 